In [1]:
import os

import dbf
import pandas as pd
from dbfread import DBF

In [2]:
path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
name_projects = "tmp1"


# name_file = r"equip.DBF"
# name_file = r"pgdynobj.DBF"
# filename = os.path.join(path_projects, name_projects, name_file)


class TypeDevice:
    P3U30 = 'P3U30'
    P3F30 = 'P3F30'
    P3T32 = 'P3T32'


class ProjectBases:

    def __init__(self, path_project):
        self.path_project = path_project
        self.base_equip = BaseProject(os.path.join(self.path_project, 'equip.DBF'))
        self.base_io = BaseProject(os.path.join(self.path_project, 'units.DBF'))
        self.base_tag = BaseProject(os.path.join(self.path_project, 'variable.DBF'))


class BaseProject:

    def __init__(self, file_name):
        self.filename = file_name

    def get_dbf(self):
        db = None
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def to_dbf(self, params_list):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def dataframe(self):
        return pd.DataFrame(DBF(self.filename, encoding='cp1251'))


class P3U30:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.name = 'P3U30'
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)


class Project:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)

    def create_equip(self, df):
        params_list = []

        for i in range(len(df)):
            dev = df.iloc[i]

            values = {'NAME': f"{dev['name']}_{dev['number']}_{dev['device']}",
                      'CLUSTER': self.cluster,
                      'TYPE': dev['device'],
                      'AREA': '',
                      'LOCATION': '',
                      'COMMENT': f"Защита ячейки {dev['number']}",
                      'CUSTOM1': f"{dev['io']}\\",
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'IODEVICE': dev['io'],
                      'PAGE': '',
                      'HELP': '',
                      'DEFSTATE': '',
                      'SCHEDULED': '',
                      'TAGPREFIX': 'Relay/',
                      'TAGGENLINK': '',
                      'LINKED': '',
                      'EDITCODE': '',
                      'PARAM': '',
                      'COMPOSITE': '',
                      'REFERENCE': '',
                      'DEVSCHED': '',
                      'SCHEDID': '',
                      'ALIAS': '',
                      'CONTENT': '',
                      'HIDDEN': ''}
            params_list.append(values)

        self.base.base_equip.to_dbf(params_list)

    def create_io(self, df):
        params_list = []

        df_tmp = self.base.base_io.dataframe()
        try:
            numb = int(df_tmp.iloc[-1]['NUMBER'])
        except:
            numb = 1

        # with dbf.Table(self.get_path_file(self.name_files['IO']), codepage='cp1251') as tb:
        #     tb.open(mode=dbf.READ_ONLY)
        #     numb = int(tb[-1]['NUMBER'])

        for i in range(len(df)):
            dev = df.iloc[i]
            numb += 1

            values = {'SERVER': self.server,
                      'NAME': dev['name'],
                      'NUMBER': f'{numb}',
                      'ADDR': dev['addr'],
                      'PROTOCOL': dev['protocol'],
                      'PORT': dev['port'],
                      'MODE': 'Primary',
                      'LOGWRITE': '',
                      'LOGREAD': '',
                      'CACHE': '',
                      'CACHETIME': '',
                      'COMMENT': '',
                      'PROTOCOLID': '',
                      'LINKED': '',
                      'EXTERNDB': '',
                      'DRIVERID': '',
                      'CONNSTRING': '',
                      'REFRESH': '',
                      'TAGPREFIX': '',
                      'LASTUPDATE': '',
                      'REMOTE': '',
                      'REMOTEWRIT': '',
                      'TIME': '',
                      'PERIOD': '',
                      'PHONE': '',
                      'CALLERID': '',
                      'ONCONNECT': '',
                      'ONHANGUP': '',
                      'ONBROWSE': '',
                      'LIVEUPDATE': '',
                      'LASTVARMOD': '',
                      'MEMORY': 'FALSE',
                      'PERSIST': '',
                      'TAGGEN': '',
                      'TAGGENTEMP': '',
                      'PRIORITY': '1',
                      'BGPOLL': '',
                      'BGPOLLRATE': '',
                      'PERSFREQ': '',
                      'PERSPATH': '',
                      'MINUPDATE': '',
                      'STALEPERIO': '',
                      'READONLY': '',
                      'EXCLUSIVE': ''}
            params_list.append(values)

        self.base.base_io.to_dbf(params_list)

    def create_tag(self, df):
        params_list = []

        for i in range(len(df)):
            dev = df.iloc[i]

            values = {'NAME': '',
                      'TYPE': '',
                      'UNIT': '',
                      'ADDR': '',
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': '',
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': '',
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}
            params_list.append(values)

        self.base.base_io.to_dbf(params_list)

    def get_path_file(self, name_file):
        return os.path.join(self.path_project, name_file)

In [ ]:
class ItemIEC61850:
    def __init__(self, name_device, node, attribs, type_attr, fs):
        self.node = node
        self.attribs = attribs
        self.attribs_str = '\\'.join(self.attribs)
        self.type_attr = type_attr
        self.fs = fs

        self.name_tag = fr"{self.node}\{self.attribs_str}"
        self.addr = ''



In [134]:
project = Project('tmp1')
plc = 'PLC_C264_TU'

df_dev = pd.read_excel('ListDevice.xlsx')

df_dev['io'] = plc
# df_dev['io'] = df_dev.apply(lambda x: f"{x['name']}_{x['number']}", axis=1)

protocol = 'IEC61850N'
df_io = pd.DataFrame([{'name': plc, 'addr': '', 'protocol': protocol, 'port': 'PORT1_BOARD1'}])
if protocol:
    df_io['addr'] = f"[RUN]:conf61850_{plc}"

project.create_io(df_io)
project.create_equip(df_dev)


In [54]:
project = Project('tmp1')
plc = 'PLC_C264_TU'

df_dev = pd.read_excel('ListDevice.xlsx', sheet_name='P3F30_61850')
df_dev

,node,atrribs,type,fc,comment
0,VO3GGIO99,Ind/t,STRING,ST,автомат кіл 3Uo вимкнено (час)
1,VO3GGIO99,Ind/stVal,STRING,ST,автомат кіл 3Uo вимкнено (режим)
2,VO4GGIO100,Ind/stVal,STRING,ST,автомат оперативних кіл комірки вимкнено (режим)
3,VO4GGIO100,Ind/t,STRING,ST,автомат оперативних кіл комірки вимкнено (час)
4,Obj2CSWI2,Pos/stVal,STRING,ST,положення викотного елементу (режим)
...,...,...,...,...,...
61,U3ppMMXU5,PPV/phsBC/t,STRING,MX,напруга фаз ВC (час)
62,U3ppMMXU5,PPV/phsCA/cVal/mag,STRING,MX,напруга фаз CA (вимірювання)
63,U3ppMMXU5,PPV/phsCA/t,STRING,MX,напруга фаз CA (час)
64,Io1MMXU11,A/neut/cVal/mag,STRING,MX,струм 3Io (вимірювання)


In [53]:
xll = pd.ExcelFile('ListDevice.xlsx')

In [7]:
project = Project('Proj16850_04112021')
df = project.base.base_tag.dataframe()
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\Obj3CSWI3\Pos\Oper\ctlVal,DIGITAL,C264,Relay/Obj3CSWI3$CO$Pos$Oper$ctlVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj3CSWI3\Pos\Oper\ctlVal
1,Cell_2\Obj2CSWI2\Pos\stVal,STRING,C264,Relay/Obj2CSWI2$ST$Pos$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
2,Cell_2\Obj2CSWI2\Pos\t,STRING,C264,Relay/Obj2CSWI2$ST$Pos$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\t
3,Cell_2\local_Obj2CSWI2_Pos_stVal,INT,Internal,,,,,,,,...,Cell_2_P3U30,,,,,,,,,local_Obj2CSWI2_Pos_stVal
4,Cell_2\Obj2CSWI2\Pos\Oper\ctlVal,DIGITAL,C264,Relay/Obj2CSWI2$CO$Pos$Oper$ctlVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\Oper\ctlVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Cell_3\U3ppMMXU5\PPV\phsBC\t,STRING,C264,Cell_3/U3ppMMXU5$MX$PPV$phsBC$t,,,,,,,...,Cell_3_P3F30,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
99,Cell_3\U3ppMMXU5\PPV\phsCA\cVal\mag,STRING,C264,Cell_3/U3ppMMXU5$MX$PPV$phsCA$cVal$mag,,,,,,,...,Cell_3_P3F30,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal\mag
100,Cell_3\U3ppMMXU5\PPV\phsCA\t,STRING,C264,Cell_3/U3ppMMXU5$MX$PPV$phsCA$t,,,,,,,...,Cell_3_P3F30,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
101,Cell_3\Io1MMXU11\A\neut\cVal\mag,STRING,C264,Cell_3/Io1MMXU11$MX$A$neut$cVal$mag,,,,,,,...,Cell_3_P3F30,,,,,,,,,Io1MMXU11\A\neut\cVal\mag


In [39]:
(df[df['EQUIP'] == 'Cell_3_P3F30']['ITEM'].apply(lambda x: x.split('\\')[0])).values

array(['VO3GGIO99', 'VO3GGIO99', 'VO4GGIO100', 'VO4GGIO100', 'Obj2CSWI2',
       'Obj2CSWI2', 'Obj3CSWI3', 'Obj3CSWI3', 'local_Obj2CSWI2_Pos_stVal',
       'Obj2CSWI2', 'Obj2CSWI2', 'Obj3CSWI3', 'Obj3CSWI3', 'ReLaGGIO141',
       'ReLaGGIO141', 'VO5GGIO101', 'VO5GGIO101', 'LLN0', 'Obj1CSWI1',
       'Obj1CSWI1', 'Obj1CSWI1', 'Obj1CSWI1', 'LO01GGIO77', 'LO01GGIO77',
       'LO01GGIO78', 'LO01GGIO78', 'LO01GGIO79', 'LO01GGIO79',
       'VO2GGIO98', 'VO2GGIO98', 'Obj3CSWI3', 'Obj3CSWI3', 'OC3PTOC3',
       'OC3PTOC3', 'OC3PTOC3', 'OC3PTOC3', 'OC1PTOC1', 'OC1PTOC1',
       'OC1PTOC1', 'OC1PTOC1', 'OC2PTOC2', 'OC2PTOC2', 'ArcM1PIOC5',
       'ArcM1PIOC5', 'EF2PTOC5', 'EF2PTOC5', 'VO6GGIO102', 'VO6GGIO102',
       'LO20GGIO96', 'LO20GGIO96', 'CBFPPIOC4', 'CBFPPIOC4', 'I3pMMXU1',
       'I3pMMXU1', 'I3pMMXU1', 'I3pMMXU1', 'I3pMMXU1', 'I3pMMXU1',
       'U3ppMMXU5', 'U3ppMMXU5', 'U3ppMMXU5', 'U3ppMMXU5', 'U3ppMMXU5',
       'U3ppMMXU5', 'Io1MMXU11', 'Io1MMXU11'], dtype=object)

In [40]:
sr = df[df['EQUIP'] == 'Cell_3_P3F30']['ITEM'].apply(lambda x: x.split('\\')[1:])
sr.apply(lambda x: '/'.join(x))

37                  Ind/t
38              Ind/stVal
39              Ind/stVal
40                  Ind/t
41              Pos/stVal
              ...        
98            PPV/phsBC/t
99     PPV/phsCA/cVal/mag
100           PPV/phsCA/t
101       A/neut/cVal/mag
102              A/neut/t
Name: ITEM, Length: 66, dtype: object

In [42]:
df[df['EQUIP'] == 'Cell_3_P3F30']['TYPE']

37     STRING
38     STRING
39     STRING
40     STRING
41     STRING
        ...  
98     STRING
99     STRING
100    STRING
101    STRING
102    STRING
Name: TYPE, Length: 66, dtype: object

In [41]:
df[df['EQUIP'] == 'Cell_3_P3F30']['ADDR'].apply(lambda x: x.split('$')[1] if x else '')

37     ST
38     ST
39     ST
40     ST
41     ST
       ..
98     MX
99     MX
100    MX
101    MX
102    MX
Name: ADDR, Length: 66, dtype: object

In [43]:
df[df['EQUIP'] == 'Cell_3_P3F30']['COMMENT']

37                        автомат кіл 3Uo вимкнено (час)
38                      автомат кіл 3Uo вимкнено (режим)
39      автомат оперативних кіл комірки вимкнено (режим)
40        автомат оперативних кіл комірки вимкнено (час)
41                  положення викотного елементу (режим)
                             ...                        
98                                  напруга фаз ВC (час)
99                          напруга фаз CA (вимірювання)
100                                 напруга фаз CA (час)
101                              струм 3Io (вимірювання)
102                                      струм 3Io (час)
Name: COMMENT, Length: 66, dtype: object

In [ ]:
project.create_io('Cell', 5)

In [106]:
df1 = pd.DataFrame([{'name': 'C264', 'addr': '[RUN]:conf61850_C264', 'protocol': 'IEC61850N', 'port': 'PORT1_BOARD1'}])
df1

,name,addr,protocol,port
0,C264,[RUN]:conf61850_C264,IEC61850N,PORT1_BOARD1


In [123]:
df = pd.read_excel('ListDevice.xlsx')
df['io'] = 'C264'
# df['io'] = df.apply(lambda x: f"{x['name']}_{x['number']}", axis=1)

,name,number,device,io
0,RZA_T,1,P3T32,RZA_T_1
1,RZA_T,2,P3T32,RZA_T_2
2,Cell_TH_6kV,2,P3U30,Cell_TH_6kV_2
3,Cell_LN_6kV,3,P3F30,Cell_LN_6kV_3
4,Cell_LN_6kV,4,P3F30,Cell_LN_6kV_4
5,Cell_IN_6kV,5,P3F30,Cell_IN_6kV_5
6,Cell_LN_6kV,6,P3F30,Cell_LN_6kV_6
7,Cell_LN_6kV,7,P3F30,Cell_LN_6kV_7
8,Cell_LN_6kV,8,P3F30,Cell_LN_6kV_8
9,Cell_LN_6kV,9,P3F30,Cell_LN_6kV_9


In [108]:
df['io'] = 'C264'
df

,name,number,device,io
0,RZA_T,1,P3T32,C264
1,RZA_T,2,P3T32,C264
2,Cell_TH_6kV,2,P3U30,C264
3,Cell_LN_6kV,3,P3F30,C264
4,Cell_LN_6kV,4,P3F30,C264
5,Cell_IN_6kV,5,P3F30,C264
6,Cell_LN_6kV,6,P3F30,C264
7,Cell_LN_6kV,7,P3F30,C264
8,Cell_LN_6kV,8,P3F30,C264
9,Cell_LN_6kV,9,P3F30,C264


In [4]:
project = Project('ExampleSA')
project.base.base_tag.dataframe()

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,ITEM,HISTORIAN,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8
0,SA_WindCompass_WindSpeed,REAL,Internal,,0,100,0,100,,,...,WindSpeed,,,,,,,,,
1,SA_WindCompass_WindSpeedAvg,REAL,Internal,,0,100,0,100,,,...,WindSpeedAvg,,,,,,,,,
2,SA_WindCompass_PRHigh,REAL,Internal,,0,100,0,100,,,...,PRHigh,,,,,,,,,
3,SA_WindCompass_PRLow,REAL,Internal,,0,100,0,100,,,...,PRLow,,,,,,,,,
4,SA_WindCompass_WindSpeedThreshold,REAL,Internal,,0,100,0,100,,,...,WindSpeedThreshold,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,TopMilk_AC02_LLEnableBypass,DIGITAL,PLC,,,,,,,,...,LLEnableBypass,,,,,,,,,
2449,TopMilk_LS007_HHPMP04BypassCommand,DIGITAL,PLC,,,,,,,,...,HHPMP04BypassCommand,,,,,,,,,
2450,TopMilk_LS007_HHPMP04RemoveBypassCommand,DIGITAL,PLC,,,,,,,,...,HHPMP04RemoveBypassCommand,,,,,,,,,
2451,TopMilk_LS007_HHPMP04BypassStatus,DIGITAL,Cicode,"Bypass_GetValue(""Company.TopMilk.Mixing.LS07.H...",,,,,,,...,HHPMP04BypassStatus,,,,,,,,,


In [97]:
df = project.base.base_io.dataframe()

In [51]:
import re

In [94]:
s = 'Cell_2_P3U30_45'
re.findall(r'.*_(\S+)', s)[-1]

'45'

In [40]:
x = BaseProject(os.path.join(r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User", 'tmp1', 'equip.DBF'))
x.dataframe()

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,Cell_2_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_2\,,,,...,,,,,,,,,,
1,Cell_3_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_3\,,,,...,,,,,,,,,,
2,Cell_4_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_4\,,,,...,,,,,,,,,,
3,Cell_5_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_5\,,,,...,,,,,,,,,,
4,Cell_6_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_6\,,,,...,,,,,,,,,,
5,Cell_7_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_7\,,,,...,,,,,,,,,,
6,Cell_8_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_8\,,,,...,,,,,,,,,,
7,Cell_9_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_9\,,,,...,,,,,,,,,,
8,Cell_10_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_10\,,,,...,,,,,,,,,,
9,Cell_11_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_11\,,,,...,,,,,,,,,,


In [137]:
dd = dbf.Table(os.path.join(r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User", 'tmp1', 'variable.DBF'),
               codepage='cp1251')
dd.open(mode=dbf.READ_ONLY)
x = dict(zip(dd.field_names, ['' for _ in range(len(dd.field_names))]))
dd.close()
x

{'NAME': '',
 'TYPE': '',
 'UNIT': '',
 'ADDR': '',
 'RAW_ZERO': '',
 'RAW_FULL': '',
 'ENG_ZERO': '',
 'ENG_FULL': '',
 'ENG_UNITS': '',
 'FORMAT': '',
 'COMMENT': '',
 'EDITCODE': '',
 'LINKED': '',
 'OID': '',
 'REF1': '',
 'REF2': '',
 'DEADBAND': '',
 'CUSTOM': '',
 'TAGGENLINK': '',
 'CLUSTER': '',
 'HISTORIAN': '',
 'EQUIP': '',
 'CUSTOM1': '',
 'CUSTOM2': '',
 'CUSTOM3': '',
 'CUSTOM4': '',
 'CUSTOM5': '',
 'CUSTOM6': '',
 'CUSTOM7': '',
 'CUSTOM8': '',
 'ITEM': ''}

In [27]:
st = ''
for t in dd.field_names:
    st += f"self.{t} = ''\n"
print(st)

self.NAME = ''
self.CLUSTER = ''
self.TYPE = ''
self.AREA = ''
self.LOCATION = ''
self.COMMENT = ''
self.CUSTOM1 = ''
self.CUSTOM2 = ''
self.CUSTOM3 = ''
self.CUSTOM4 = ''
self.CUSTOM5 = ''
self.CUSTOM6 = ''
self.CUSTOM7 = ''
self.CUSTOM8 = ''
self.IODEVICE = ''
self.PAGE = ''
self.HELP = ''
self.DEFSTATE = ''
self.SCHEDULED = ''
self.TAGPREFIX = ''
self.TAGGENLINK = ''
self.LINKED = ''
self.EDITCODE = ''
self.PARAM = ''
self.COMPOSITE = ''
self.REFERENCE = ''
self.DEVSCHED = ''
self.SCHEDID = ''
self.ALIAS = ''
self.CONTENT = ''
self.HIDDEN = ''



In [31]:
class BaseProject:

    def __init__(self, file_name):
        self.filename = filename


In [32]:
x = Clss()

In [34]:
x.filename = 'jfjdklj'
x.__dict__

{'NAME': '',
 'CLUSTER': '',
 'TYPE': '',
 'AREA': '',
 'LOCATION': '',
 'COMMENT': '',
 'CUSTOM1': '',
 'CUSTOM2': '',
 'CUSTOM3': '',
 'CUSTOM4': '',
 'CUSTOM5': '',
 'CUSTOM6': '',
 'CUSTOM7': '',
 'CUSTOM8': '',
 'IODEVICE': '',
 'PAGE': '',
 'HELP': '',
 'DEFSTATE': '',
 'SCHEDULED': '',
 'TAGPREFIX': '',
 'TAGGENLINK': '',
 'LINKED': '',
 'EDITCODE': '',
 'PARAM': '',
 'COMPOSITE': '',
 'REFERENCE': '',
 'DEVSCHED': '',
 'SCHEDID': '',
 'ALIAS': '',
 'CONTENT': '',
 'HIDDEN': '',
 'filename': 'jfjdklj'}

In [77]:
dd.open(mode=dbf.READ_ONLY)

dbf.Table('c:\\ProgramData\\AVEVA\\Citect SCADA 2018 R2\\User\\tmp1\\units.DBF', status=<DbfStatus.READ_ONLY: 1>)

In [12]:
dd.close()

In [68]:
project.show_io()

,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [57]:
project = Project('tmp1')
project.write_io()

ValueError: invalid literal for int() with base 10: '        '

Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_39_64.pyx", line 1034, in _pydevd_bundle.pydevd_cython_win32_39_64.PyDBFrame.trace_dispatch
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [20]:
# Удаление строки
rec = table[-1]
dbf.delete(rec)
table.pack()

In [11]:
name_file = r"units.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df


,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [10]:
name_file = r"variable.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\VO3GGIO99\Ind\t,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\t
1,Cell_2\VO3GGIO99\Ind\stVal,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\stVal
2,Cell_2\VO4GGIO100\Ind\stVal,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\stVal
3,Cell_2\VO4GGIO100\Ind\t,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\t
4,Cell_2\Obj2CSWI2\Pos\stVal,STRING,Cell_2,Relay/Obj2CSWI2$ST$Pos$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,Cell_20\U3ppMMXU5\PPV\phsBC\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsBC$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
699,Cell_20\U3ppMMXU5\PPV\phsCA\cVal,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal
700,Cell_20\U3ppMMXU5\PPV\phsCA\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
701,Cell_20\UovMMXU1\PhV\neut\cVal,STRING,Cell_20,Relay/UovMMXU1$MX$PhV$neut$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,UovMMXU1\PhV\neut\cVal


In [7]:
name_file = r"equip.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,Cell_2_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_2\,,,,...,,,,,,,,,,
1,Cell_3_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_3\,,,,...,,,,,,,,,,
2,Cell_4_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_4\,,,,...,,,,,,,,,,
3,Cell_5_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_5\,,,,...,,,,,,,,,,
4,Cell_6_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_6\,,,,...,,,,,,,,,,
5,Cell_7_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_7\,,,,...,,,,,,,,,,
6,Cell_8_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_8\,,,,...,,,,,,,,,,
7,Cell_9_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_9\,,,,...,,,,,,,,,,
8,Cell_10_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_10\,,,,...,,,,,,,,,,
9,Cell_11_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_11\,,,,...,,,,,,,,,,
